In [2]:
# Change the Jupyter kernel's executable
%config IPCompleter.use_jedi = False
%config IPCompleter.use_matplotlib = False
%config IPCompleter.greedy = True

# Change the kernel's executable to the virtual environment's Python
import sys
sys.executable = '/Users/nikhil/My Computer/A Year 2/DSA/python/image processing/Assignment 2/venv/bin/python3'

# Verify the Python executable path
sys.executable

import cv2
print(cv2.__version__)




/Users/nikhil/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3493: UserWarning: Config option `use_matplotlib` not recognized by `IPCompleter`.
  if await self.run_code(code, result, async_=asy):


4.8.1


In [1]:
import numpy as np
import cv2
import glob
import argparse

In [2]:
def save_coefficients(mtx, dist, path):
    """ Save the camera matrix and the distortion coefficients to given path/file. """
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write("K", mtx)
    cv_file.write("D", dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()
def load_coefficients(path):
    """ Loads camera matrix and distortion coefficients. """
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode("K").mat()
    dist_matrix = cv_file.getNode("D").mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]

In [3]:
import numpy as np
import cv2
import glob

# Global variables to store calibration results
calibration_results = None

def calibrate(dirpath, prefix, image_format, square_size, width=9, height=6):
    """ Apply camera calibration operation for images in the given directory path. """
    global calibration_results

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real-world space
    imgpoints = []  # 2d points in the image plane.

    if dirpath[-1:] == '/':
        dirpath = dirpath[:-1]

    images = glob.glob(dirpath + '/' + prefix + '*.' + image_format)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), None)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (width, height), corners2, ret)

            # Calibrate camera for each image
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp], [corners2], gray.shape[::-1], None, None)

            # Store the calibration results in the global variable
            calibration_results = [ret, mtx, dist, rvecs, tvecs]

    return calibration_results

# Example usage
dirpath = '/Users/nikhil/My Computer/A Year 2/DSA/python/image processing/calibratediary'
prefix = 'img'
image_format = 'png'
square_size = 25.0  # Adjust the square size based on your calibration pattern
width = 10
height = 7

calibration_results = calibrate(dirpath, prefix, image_format, square_size, width, height)
print(calibration_results)

save_coefficients(calibration_results[1], calibration_results[2], "calibration.yaml")

# Check if calibration was successful before unpacking results
if calibration_results is not None:
    ret, mtx, dist, rvecs, tvecs = calibration_results
    # Access calibration results if needed
    print("Calibration successful.")
else:
    print("Calibration failed.")





[0.18100203605511817, array([[1.54796962e+03, 0.00000000e+00, 9.29596692e+02],
       [0.00000000e+00, 1.54394442e+03, 5.69504179e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), array([[ 0.06852545,  0.02704767,  0.00174481, -0.00469339, -0.24183901]]), (array([[ 0.03566208],
       [ 0.35416381],
       [-1.53153193]]),), (array([[ 52.49556764],
       [108.13678615],
       [508.43002555]]),)]
Calibration successful.


In [12]:
# Load the calibration coefficients later if needed
loaded_coefficients = load_coefficients("calibration.yaml")

# Assign the loaded coefficients to mtx and dist
mtx, dist = loaded_coefficients

from cv2 import aruco

def track(matrix_coefficients, distortion_coefficients):
    cap = cv2.VideoCapture(0)  # Change camera index if needed

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    # Define ArUco dictionary and parameters
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to capture a frame. Exiting...")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect markers
        markerCorners, markerIds, rejectedCandidates = detector.detectMarkers(gray)

        if markerIds is not None and len(markerIds) > 0:
            # Draw detected markers
            frame = cv2.aruco.drawDetectedMarkers(frame, markerCorners, markerIds)

        # Display the resulting frame
        cv2.imshow('frame', frame)

        key = cv2.waitKey(3) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the track function with calibration results
track(mtx,dist)



    

Failed to capture a frame. Exiting...


In [4]:


def markers(matrix_coefficients, distortion_coefficients):
    cap = cv2.VideoCapture(0)  # Change camera index if needed

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    # Define ArUco dictionary and parameters
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)
    
    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to capture a frame. Exiting...")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect markers
        markerCorners, markerIds, rejectedCandidates = detector.detectMarkers(gray)

        if markerIds is not None and len(markerIds) > 0:
            # Draw bounding box for the detected marker
            frame = cv2.aruco.drawDetectedMarkers(frame, markerCorners, markerIds)

            # Calculate pose information for the detected marker
            rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(markerCorners, 0.05, matrix_coefficients, distortion_coefficients)

            # Draw pose axes
            frame = cv2.aruco.drawAxis(frame, matrix_coefficients, distortion_coefficients, rvec, tvec, 0.1)

            # Compute the center of the marker
            xc = int((markerCorners[0][0][0][0] + markerCorners[0][0][2][0]) / 2)
            yc = int((markerCorners[0][0][0][1] + markerCorners[0][0][2][1]) / 2)

            # Display the center coordinates (xc, yc)
            print(f"Center Coordinates: ({xc}, {yc})")

        # Display the resulting frame
        cv2.imshow('frame', frame)

        key = cv2.waitKey(3) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage with calibration results
mtx, dist = load_coefficients("calibration.yaml")
markers(mtx, dist)


AttributeError: module 'cv2.aruco' has no attribute 'estimatePoseSingleMarkers'

In [4]:


def markers(s, mtx, dist):
    # Load the image
    img = cv2.imread(s)

    # Define ArUco dictionary and parameters
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)

    # Detect markers in the image
    markerCorners, markerIds, rejectedCandidates = detector.detectMarkers(img)

    # Check if any markers are detected
    if markerIds is not None and len(markerIds) > 0:
        # Draw bounding box for the detected marker
        cv2.aruco.drawDetectedMarkers(img, markerCorners, markerIds)

        # Estimate pose of the detected marker
        rvecs, tvecs,_= cv2.aruco.estimatePoseSingleMarkers(markerCorners, 0.02, mtx, dist)

        # Draw pose axes for each detected marker
        for i in range(len(markerIds)):
            cv2.aruco.drawAxis(img, mtx, dist, rvecs[i], tvecs[i], 0.01)

        # Get translation and rotation vectors for the first marker (assuming only one marker is present)
        translation_vector = tvecs[0].flatten()
        rotation_vector = rvecs[0].flatten()

        # Compute center of the marker
        xc, yc = np.mean(markerCorners[0][0], axis=0)

        # Display the image with the bounding box and pose axes
        cv2.imshow('Detected Markers', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Return relevant information
        return xc, yc, translation_vector, rotation_vector
    else:
        print("No markers detected in the image.")
        return None


image_path = 'img0.png'

loaded_coefficients = load_coefficients("calibration.yaml")
mtx, dist = loaded_coefficients
result = markers(image_path, mtx, dist)

if result is not None:
    xc, yc, translation_vector, rotation_vector = result
    print("Center of Marker (xc, yc):", xc, yc)
    print("Translation Vector:", translation_vector)
    print("Rotation Vector:", rotation_vector)


AttributeError: module 'cv2.aruco' has no attribute 'estimatePoseSingleMarkers'

In [10]:
import cv2
import numpy as np

def markers(s, mtx, dist):
    # Load the image
    img = cv2.imread(s)

    # Define ArUco dictionary and parameters
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
    parameters = cv2.aruco.DetectorParameters()
    parameters.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_CONTOUR
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)

    # Detect markers in the image
    markerCorners, markerIds, rejectedCandidates = detector.detectMarkers(img)

    # Check if any markers are detected
    if markerIds is not None and len(markerIds) > 0:
        # Draw bounding box for the detected marker
        cv2.aruco.drawDetectedMarkers(img, markerCorners, markerIds)

        # Estimate pose of the detected marker
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(markerCorners, 0.02, None, None, mtx, dist, cameraMatrix=None, markerLength=1.0, parameters=parameters)

        # Draw pose axes for each detected marker
        for i in range(len(markerIds)):
            cv2.aruco.drawAxis(img, mtx, dist, rvecs[i], tvecs[i], 0.01)

        # Get translation and rotation vectors for the first marker (assuming only one marker is present)
        translation_vector = tvecs[0].flatten()
        rotation_vector = rvecs[0].flatten()

        # Compute center of the marker
        xc, yc = np.mean(markerCorners[0][0], axis=0)

        # Display the image with the bounding box and pose axes
        cv2.imshow('Detected Markers', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        # Return relevant information
        return xc, yc, translation_vector, rotation_vector
    else:
        print("No markers detected in the image.")
        return None


image_path = 'img0.png'

# Assuming the load_coefficients function is defined and returns loaded_coefficients
loaded_coefficients = load_coefficients("calibration.yaml")
mtx, dist = loaded_coefficients
result = markers(image_path, mtx, dist)

if result is not None:
    xc, yc, translation_vector, rotation_vector = result
    print("Center of Marker (xc, yc):", xc, yc)
    print("Translation Vector:", translation_vector)
    print("Rotation Vector:", rotation_vector)


AttributeError: module 'cv2.aruco' has no attribute 'estimatePoseSingleMarkers'